In [1]:
import java.io.File
abstract class Problem<T>(private val filename: String) {
    abstract val testResult: T
    protected var isTesting: Boolean = true
        private set
    fun run() {
        val testOutput = File("testData/$filename").useLines { solve(it.toList()) }
        if (testOutput != testResult) {
            throw IllegalStateException("Test output doesn't match expected result \n Actual: $testOutput \n Expected: $testResult")
        }
        println("Test results: Expected=$testResult Actual=$testOutput")

        isTesting = false
        val actualOutput = File("data/$filename").useLines { solve(it.toList()) }
        println("Solution: $actualOutput")
    }
    abstract fun solve(lines: List<String>): T
}

## Day 8
### Part 1
Equipped with a new understanding of teleporter maintenance, you confidently step onto the repaired teleporter pad.

You rematerialize on an unfamiliar teleporter pad and find yourself in a vast underground space which contains a giant playground!

Across the playground, a group of Elves are working on setting up an ambitious Christmas decoration project. Through careful rigging, they have suspended a large number of small electrical junction boxes.

Their plan is to connect the junction boxes with long strings of lights. Most of the junction boxes don't provide electricity; however, when two junction boxes are connected by a string of lights, electricity can pass between those two junction boxes.

The Elves are trying to figure out which junction boxes to connect so that electricity can reach every junction box. They even have a list of all of the junction boxes' positions in 3D space (your puzzle input).

For example:
```
162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
```
This list describes the position of 20 junction boxes, one per line. Each position is given as `X,Y,Z` coordinates. So, the first junction box in the list is at `X=162`, `Y=817`, `Z=812`.

To save on string lights, the Elves would like to focus on connecting pairs of junction boxes that are as close together as possible according to straight-line distance. In this example, the two junction boxes which are closest together are `162,817,812` and `425,690,689`.

By connecting these two junction boxes together, because electricity can flow between them, they become part of the same circuit. After connecting them, there is a single circuit which contains two junction boxes, and the remaining 18 junction boxes remain in their own individual circuits.

Now, the two junction boxes which are closest together but aren't already directly connected are `162,817,812` and `431,825,988`. After connecting them, since `162,817,812` is already connected to another junction box, there is now a single circuit which contains three junction boxes and an additional 17 circuits which contain one junction box each.

The next two junction boxes to connect are `906,360,560` and `805,96,715` . After connecting them, there is a circuit containing 3 junction boxes, a circuit containing 2 junction boxes, and 15 circuits which contain one junction box each.

The next two junction boxes are `431,825,988` and `425,690,689`. Because these two junction boxes were already in the same circuit, nothing happens!

This process continues for a while, and the Elves are concerned that they don't have enough extension cables for all these circuits. They would like to know how big the circuits will be.

After making the ten shortest connections, there are 11 circuits: one circuit which contains 5 junction boxes, one circuit which contains 4 junction boxes, two circuits which contain 2 junction boxes each, and seven circuits which each contain a single junction box. Multiplying together the sizes of the three largest circuits (5, 4, and one of the circuits of size 2) produces 40.

Your list contains many junction boxes; connect together the 1000 pairs of junction boxes which are closest together. Afterward, what do you get if you multiply together the sizes of the three largest circuits?

In [2]:
class DayEightPartOne(override val testResult: Int = 40) : Problem<Int>("day8.txt") {
    override fun solve(lines: List<String>): Int {
        val coords = lines.map { line ->
            val items = line.split(",")
            Coord(x = items[0].toLong(), y = items[1].toLong(), z = items[2].toLong())
        }
        val mappings = coords.mapIndexed { index, coord ->
            Mapping(coord).also {
                it.addDistances(coords.subList(index + 1, coords.size))
            }
        }
        val groups: MutableList<MutableSet<Coord>> = mutableListOf()
        repeat(if (isTesting) 10 else 1000) {
            val min = mappings.minBy { it.minDistance()?.value ?: Long.MAX_VALUE }
            val other = min.minDistance()!!
            val matchingGroups = groups.filter { it.contains(min.coord) || it.contains(other.key) }
            when (matchingGroups.size) {
                0 -> {
                    groups.add(mutableSetOf(min.coord, other.key))
                }
                1 -> {
                    groups[groups.indexOf(matchingGroups[0])].let {
                        it.add(min.coord)
                        it.add(other.key)
                    }
                }
                else -> {
                    val newGroup = matchingGroups.flatMap { it }.toMutableSet()
                    newGroup.add(min.coord)
                    newGroup.add(other.key)
                    groups.removeAll(matchingGroups)
                    groups.add(newGroup)
                }
            }
            min.remove(other.key)
        }

        var out = 1
        groups.sortedByDescending { it.size }.take(3).forEach { out *= it.size }
        return out
    }

    data class Coord(val x: Long, val y: Long, val z: Long) {
        fun distanceTo(other: Coord): Long = (this.x - other.x).squared() + (this.y - other.y).squared() + (this.z - other.z).squared()
        private fun Long.squared() = this * this
        override fun toString(): String = "($x, $y, $z)"
    }

    data class Mapping(val coord: Coord) {
        private val mappings: MutableMap<Coord, Long> = mutableMapOf()
        fun addDistances(coords: List<Coord>) {
            coords.forEach { other ->
                if (coord != other) {
                    mappings[other] = coord.distanceTo(other)
                }
            }
        }

        fun remove(coord: Coord) {
            mappings.remove(coord)
        }

        fun minDistance(): Map.Entry<Coord, Long>? = if (mappings.isNotEmpty()) mappings.minBy { it.value } else null
    }
}

DayEightPartOne().run()

Test results: Expected=40 Actual=40
Solution: 135169


### Part 2
The Elves were right; they definitely don't have enough extension cables. You'll need to keep connecting junction boxes together until they're all in one large circuit.

Continuing the above example, the first connection which causes all of the junction boxes to form a single circuit is between the junction boxes at 216,146,977 and 117,168,530. The Elves need to know how far those junction boxes are from the wall so they can pick the right extension cable; multiplying the X coordinates of those two junction boxes (216 and 117) produces 25272.

Continue connecting the closest unconnected pairs of junction boxes together until they're all in the same circuit. What do you get if you multiply together the X coordinates of the last two junction boxes you need to connect?

In [3]:
class DayEightPartTwo(override val testResult: Long = 25272) : Problem<Long>("day8.txt") {
    override fun solve(lines: List<String>): Long {
        val coords = lines.map { line ->
            val items = line.split(",")
            Coord(x = items[0].toLong(), y = items[1].toLong(), z = items[2].toLong())
        }
        val mappings = coords.mapIndexed { index, coord ->
            Mapping(coord).also {
                it.addDistances(coords.subList(index + 1, coords.size))
            }
        }
        val groups: MutableList<MutableSet<Coord>> = mutableListOf()
        var lastPair: Pair<Coord, Coord> = Coord(0L, 0L, 0L) to Coord(0L, 0L, 0L)
        do {
            val hasConnections = mappings.filter { it.hasConnectionsAvailable() }
            val min = hasConnections.minBy { it.minDistance()?.value ?: Long.MAX_VALUE }
            val other = min.minDistance()!!
            val matchingGroups = groups.filter { it.contains(min.coord) || it.contains(other.key) }
            when (matchingGroups.size) {
                0 -> {
                    groups.add(mutableSetOf(min.coord, other.key))
                }
                1 -> {
                    groups[groups.indexOf(matchingGroups[0])].let {
                        it.add(min.coord)
                        it.add(other.key)
                    }
                }
                else -> {
                    val newGroup = (matchingGroups[0] + matchingGroups[1]).toMutableSet()
                    newGroup.add(min.coord)
                    newGroup.add(other.key)
                    groups.removeAll(matchingGroups)
                    groups.add(newGroup)
                }
            }
            min.remove(other.key)

            lastPair = min.coord to other.key
        } while (groups[0].size != mappings.size)

        return lastPair.first.x * lastPair.second.x
    }

    data class Coord(val x: Long, val y: Long, val z: Long) {
        fun distanceTo(other: Coord): Long = (this.x - other.x).squared() + (this.y - other.y).squared() + (this.z - other.z).squared()
        private fun Long.squared() = this * this
        override fun toString(): String = "($x, $y, $z)"
    }

    data class Mapping(val coord: Coord) {
        private val mappings: MutableMap<Coord, Long> = mutableMapOf()
        fun addDistances(coords: List<Coord>) {
            coords.forEach { other ->
                if (coord != other) {
                    mappings[other] = coord.distanceTo(other)
                }
            }
        }

        fun remove(coord: Coord) {
            mappings.remove(coord)
        }

        fun minDistance(): Map.Entry<Coord, Long>? = if (mappings.isNotEmpty()) mappings.minBy { it.value } else null

        fun hasConnectionsAvailable(): Boolean = mappings.isNotEmpty()
    }
}

DayEightPartTwo().run()

Test results: Expected=25272 Actual=25272
Solution: 302133440
